In [2]:
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import os
import sys

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import append_metadata

In [ ]:
@append_metadata
def subset_to_CA(input_file_name, output_file_name, directory, export=False, varname=''):
    '''
    Subsets USGS impervious lands data to California, then uploads to AWS subset bucket
    
    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    
    Parameters
    ----------
    input_file_name: string
        Name of origional .csv file    
    output_file_name: string
        Final output as a .csv file   
    directory: string
        Directory path for output to upload to in AWS
    export: bool
        If True, runs code and uploads output file to S3
        If False, just generates metadata

    Script
    ------
    usgs_impervious_lands_subset.ipynb
    '''
    
    ## set-up for AWS  
    s3_client = boto3.client('s3')  
    bucket_name = 'ca-climate-index'  
    
    # run code if export is true, if false, nothing happens (useful when appending metadata)
    if export == True: 
        # read in raster covering CONUS
        rds = rxr.open_rasterio(input_file_name).squeeze()

        # read in CA state shapefile
        ca_shp_dir = "s3://ca-climate-index/0_map_data/ca_state/"
        ca_shp = gpd.read_file(ca_shp_dir)
        # reproject CA boundaries to the raster's CRS
        ca_shp = ca_shp.to_crs(ds.rio.crs)
        ca_bounds = ca_shp.bounds
        minx = ca_bounds.minx.values[0]
        maxx = ca_bounds.maxx.values[0]
        miny = ca_bounds.miny.values[0]
        maxy = ca_bounds.maxy.values[0]
        
        # subset raster to CA only
        ds_bnd = ds.sel(x=slice(minx, maxx),y=slice(maxy, miny))
        ds_bnd.name = "impervious_surface"
        ds_bnd = ds_bnd.to_dataset()
        ds_bnd = ds_bnd.chunk(chunks="auto")

        # ship off the new file to S3
        ds_bnd.to_zarr(output_file_name)
        print(f'{output_file_name} uploaded to AWS.')

    if export == False:
        print(f'{output_file_name} uploaded to AWS.')


In [5]:
# read in raster file
# this is saved locally rather than in S3 because it is very large! 
datpath = os.path.join(
    "/mnt", "c", "Users", "eliza", "Downloads", 
    "nlcd_2021_impervious_l48_20230630",
  "nlcd_2021_impervious_l48_20230630.img"
                      )

,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspaceType,cancellationDateOfFloodPolicy,condominiumCoverageTypeCode,construction,crsClassCode,buildingDeductibleCode,contentsDeductibleCode,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,federalPolicyFee,ratedFloodZone,hfiaaSurcharge,houseOfWorshipIndicator,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,nonProfitIndicator,numberOfFloorsInInsuredBuilding,obstructionType,occupancyType,originalConstructionDate,originalNBDate,policyCost,policyCount,policyEffectiveDate,policyTerminationDate,policyTermIndicator,postFIRMConstructionIndicator,primaryResidenceIndicator,rateMethod,regularEmergencyProgramIndicator,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy,cancellationVoidanceReasonCode,subsidizedRateType,iccPremium,reserveFundAssessment,communityProbationSurcharge,premiumPaymentIndicator,buildingReplacementCost,basicBuildingRate,additionalBuildingRate,basicContentsRate,AdditionalContentsRate,enclosureTypeCode,buildingDescriptionCode,insuranceToValueCode,postFirmVzoneIndicator,floodproofedIndicator,waitingPeriodType,rolloverTransferCode,endorsementEffectiveDate,propertyPurchaseDate,rentalPropertyIndicator,tenantIndicator,stateOwnedIndicator,disasterAssistanceCoverageRequiredCode,mandatoryPurchaseFlag,grandfatheringTypeCode,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,programTypeIndicator,mapPanelNumber,mapPanelSuffix,floodZoneCurrent,femaRegion,propertyState,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
npartitions=531,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,object,object,int64,object,object,object,int64,object,float64,int64,object,int64,int64,float64,float64,float64,int64,object,float64,object,object,object,int64,int64,object,object,int64,int64,int64,object,object,int64,object,object,int64,object,object,object,int64,object,float64,object,float64,float64,float64,float64,object,float64,float64,int64,int64,object,object,object,object,int64,int64,int64,object,int64,float64,object,object,object,int64,object,object,object,object,object,object,object,object,object,object,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
